In [0]:
!pip3 install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-emjxh_h2
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-emjxh_h2
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=19950 sha256=f4448db35ea384b32244bef38645dff8b50132ed990fb4e508076ad59a0d3cf3
  Stored in directory: /tmp/pip-ephem-wheel-cache-gw3t43ex/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import keras
import os
import random
import numpy as np
import time
from matplotlib import pyplot as plt
from keras.callbacks import LearningRateScheduler
import keras.backend as K

In [0]:
# for keras
from classification_models.keras import Classifiers

# for tensorflow.keras
# from classification_models.tfkeras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
conv_base = ResNet18((128, 128, 3), weights='imagenet', include_top= False)













44924928/44920640 [==============================] - 1s 0us/step


In [0]:
conv_base.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 128, 128, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 64, 64, 64)   9408        zero_padding2d_1[0][0]           
____________________________________________________________________________________________

In [0]:
# Load cifar using the load data function
# The data is in the shape of (Batch_size, num_channel, width, height)
# np.unique returns unique class names

(train_features, train_labels), (test_features, test_labels) =  keras.datasets.cifar10.load_data()
num_train, img_rows, img_cols, img_channels =  train_features.shape
num_test, _, _, _ =  test_features.shape
num_classes = len(np.unique(train_labels))
print(num_classes)



# # Visualization of some particular classes from the dataset
# class_names = ['airplane','automobile','bird','cat','deer',
#                'dog','frog','horse','ship','truck']
# fig = plt.figure(figsize=(8,3))
# for i in range(num_classes):
#     ax = fig.add_subplot(2, 5,1 + i, xticks=[], yticks=[])
#     # gets the all the position of a particular class
#     idx = np.where(train_labels[:]==i)[0]
#     # using the idx we can find the corresponding image arrays
#     features_idx = train_features[idx,::]
#     # selects an img num by random
#     img_num = np.random.randint(features_idx.shape[0])
#     # gets the img array by img num
#     im = features_idx[img_num]
#     ax.set_title(class_names[i])
#     plt.imshow(im)
# plt.show()

10


In [0]:
import cv2
def resize_data(data):
    data_upscaled = []
    for i, img in enumerate(data):
        large_img = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
        data_upscaled.append(large_img)
    return np.array(data_upscaled)

# resize train and  test data
train_features = resize_data(train_features)
test_features = resize_data(test_features)

In [0]:
test_features.shape

(10000, 128, 128, 3)

In [0]:
# Normalize data
train_features = preprocess_input(train_features)
test_features = preprocess_input(test_features)
# Convert class labels to binary class labels
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

In [0]:
# mean = [0.4914, 0.4822, 0.4465]
# std = [0.2023, 0.1994, 0.2010]
# train_features = train_features - mean
# train_features = train_features / std 

In [0]:
train_features.shape

(50000, 128, 128, 3)

In [0]:
# code is used from https://github.com/yu4u/cutout-random-erasing/blob/master/random_eraser.py

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    
    # def norm_data(input_img):
    #   input_img = input_img - mean
    #   input_img = input_img / std
    #   return input_img
    

    def eraser(input_img):
        # input_img = norm_data(input_img)
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

class LR_Finder(keras.callbacks.Callback):
    
    def __init__(self, start_lr=1e-5, end_lr=10, step_size=None, beta=.98):
        super().__init__()
        
        self.start_lr = start_lr
        self.end_lr = end_lr
        self.step_size = step_size
        self.beta = beta
        self.lr_mult = (end_lr/start_lr)**(1/step_size)
        
    def on_train_begin(self, logs=None):
        self.best_loss = 1e9
        self.avg_loss = 0
        self.losses, self.smoothed_losses, self.lrs, self.iterations = [], [], [], []
        self.iteration = 0
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.start_lr)
        
    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        self.iteration += 1
        
        self.avg_loss = self.beta * self.avg_loss + (1 - self.beta) * loss
        smoothed_loss = self.avg_loss / (1 - self.beta**self.iteration)
        
        # Check if the loss is not exploding
        if self.iteration>1 and smoothed_loss > self.best_loss * 4:
            self.model.stop_training = True
            return

        if smoothed_loss < self.best_loss or self.iteration==1:
            self.best_loss = smoothed_loss
        
        lr = self.start_lr * (self.lr_mult**self.iteration)
        
        self.losses.append(loss)
        self.smoothed_losses.append(smoothed_loss)
        self.lrs.append(lr)
        self.iterations.append(self.iteration)
        
        
        K.set_value(self.model.optimizer.lr, lr)  
        
    def plot_lr(self):
        plt.xlabel('Iterations')
        plt.ylabel('Learning rate')
        plt.plot(self.iterations, self.lrs)
        
    def plot(self, n_skip=10):
        plt.ylabel('Loss')
        plt.xlabel('Learning rate (log scale)')
        plt.plot(self.lrs[n_skip:-5], self.losses[n_skip:-5])
        plt.xscale('log')
        
    def plot_smoothed_loss(self, n_skip=10):
        plt.ylabel('Smoothed Losses')
        plt.xlabel('Learning rate (log scale)')
        plt.plot(self.lrs[n_skip:-5], self.smoothed_losses[n_skip:-5])
        plt.xscale('log')
        
    def plot_loss(self):
        plt.ylabel('Losses')
        plt.xlabel('Iterations')
        plt.plot(self.iterations[10:], self.losses[10:])

In [0]:
# Create datagenerate for trainingtrain_batches
datagen = keras.preprocessing.image.ImageDataGenerator(zoom_range=0.0, 
                             horizontal_flip=True,preprocessing_function=get_random_eraser(v_l=0.1, v_h=0.3))

In [0]:
# input_tensor = keras.layers.Input((32,32,3))
# x = keras.layers.UpSampling2D()(input_tensor)
# x = keras.layers.UpSampling2D()(x)
# x = keras.layers.UpSampling2D()(x)
# x = conv_base(conv_base.output)
x = keras.layers.Conv2D(10,1)(conv_base.output)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Activation(tf.nn.softmax)(x)
model = keras.models.Model(conv_base.input,x)

In [0]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 128, 128, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 64, 64, 64)   9408        zero_padding2d_1[0][0]           
____________________________________________________________________________________________

In [0]:
################### Define the callbacks ###################

base_dir = '/content/drive/My Drive/EVA/Session20'


if not (os.path.exists(os.path.join(base_dir, 'weights_pretrain'))):
    os.mkdir(os.path.join(base_dir, 'weights_pretrain'))

filepath = os.path.join('/content/drive/My Drive/EVA/Session20/weights_pretrain',
                        'weights-improvement-best.hdf5')

checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             mode='max')

In [0]:
# ##################### freeze layers for pretrain #######
for index,l in enumerate(model.layers):
  if index <  86:
    l.trainable = False
  else:
    l.trainable = True
for index,l in enumerate(model.layers):
  print(l.name,index,l.trainable)

data 0 False
bn_data 1 False
zero_padding2d_1 2 False
conv0 3 False
bn0 4 False
relu0 5 False
zero_padding2d_2 6 False
pooling0 7 False
stage1_unit1_bn1 8 False
stage1_unit1_relu1 9 False
zero_padding2d_3 10 False
stage1_unit1_conv1 11 False
stage1_unit1_bn2 12 False
stage1_unit1_relu2 13 False
zero_padding2d_4 14 False
stage1_unit1_conv2 15 False
stage1_unit1_sc 16 False
add_1 17 False
stage1_unit2_bn1 18 False
stage1_unit2_relu1 19 False
zero_padding2d_5 20 False
stage1_unit2_conv1 21 False
stage1_unit2_bn2 22 False
stage1_unit2_relu2 23 False
zero_padding2d_6 24 False
stage1_unit2_conv2 25 False
add_2 26 False
stage2_unit1_bn1 27 False
stage2_unit1_relu1 28 False
zero_padding2d_7 29 False
stage2_unit1_conv1 30 False
stage2_unit1_bn2 31 False
stage2_unit1_relu2 32 False
zero_padding2d_8 33 False
stage2_unit1_conv2 34 False
stage2_unit1_sc 35 False
add_3 36 False
stage2_unit2_bn1 37 False
stage2_unit2_relu1 38 False
zero_padding2d_9 39 False
stage2_unit2_conv1 40 False
stage2_unit2_bn

In [0]:
model.compile(optimizer = keras.optimizers.SGD(lr=.1, momentum=.9, decay=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
start = time.time()
step_per_epoch_train = len(train_features) // 128
step_per_epoch_validation = len(test_features) // 128
# Train the model
model_info = model.fit_generator(datagen.flow(train_features, train_labels, batch_size = 128), epochs = 5,
                                 steps_per_epoch = step_per_epoch_train,
                                 validation_data = (test_features, test_labels), verbose=1,
                                 validation_steps = step_per_epoch_validation,
                                 callbacks = [checkpoint])
end = time.time()

print ("Model took %0.2f seconds to train"%(end - start))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
390/390 [==============================] - 21s 54ms/step - loss: 1.5056 - acc: 0.6310 - val_loss: 1.4750 - val_acc: 0.6912

Epoch 00001: val_acc improved from -inf to 0.69120, saving model to /content/drive/My Drive/EVA/Session20/weights_pretrain/weights-improvement-best.hdf5
Epoch 2/5
390/390 [==============================] - 18s 47ms/step - loss: 0.9597 - acc: 0.6831 - val_loss: 1.0157 - val_acc: 0.7343

Epoch 00002: val_acc improved from 0.69120 to 0.73430, saving model to /content/drive/My Drive/EVA/Session20/weights_pretrain/weights-improvement-best.hdf5
Epoch 3/5
390/390 [==============================] - 18s 47ms/step - loss: 0.8826 - acc: 0.6956 - val_loss: 1.1280 - val_acc: 0.7120

Epoch 00003: val_acc did not improve from 0.73430
Epoch 4/5
390/390 [==============================] - 18s 47ms/step - loss: 0.8556 - acc: 0.7020 - val_loss: 0.9725 - val_acc: 0.7393

Epoch 000

In [0]:
# load pretrain model to do finetuning
model = keras.models.load_model('/content/drive/My Drive/EVA/Session20/weights_pretrain/weights-improvement-best.hdf5')

In [0]:
for index,l in enumerate(model.layers):
  if index <  46:
    l.trainable = False
  else:
    l.trainable = True
for index,l in enumerate(model.layers):
  print(l.name,index,l.trainable)

data 0 False
bn_data 1 False
zero_padding2d_1 2 False
conv0 3 False
bn0 4 False
relu0 5 False
zero_padding2d_2 6 False
pooling0 7 False
stage1_unit1_bn1 8 False
stage1_unit1_relu1 9 False
zero_padding2d_3 10 False
stage1_unit1_conv1 11 False
stage1_unit1_bn2 12 False
stage1_unit1_relu2 13 False
zero_padding2d_4 14 False
stage1_unit1_conv2 15 False
stage1_unit1_sc 16 False
add_1 17 False
stage1_unit2_bn1 18 False
stage1_unit2_relu1 19 False
zero_padding2d_5 20 False
stage1_unit2_conv1 21 False
stage1_unit2_bn2 22 False
stage1_unit2_relu2 23 False
zero_padding2d_6 24 False
stage1_unit2_conv2 25 False
add_2 26 False
stage2_unit1_bn1 27 False
stage2_unit1_relu1 28 False
zero_padding2d_7 29 False
stage2_unit1_conv1 30 False
stage2_unit1_bn2 31 False
stage2_unit1_relu2 32 False
zero_padding2d_8 33 False
stage2_unit1_conv2 34 False
stage2_unit1_sc 35 False
add_3 36 False
stage2_unit2_bn1 37 False
stage2_unit2_relu1 38 False
zero_padding2d_9 39 False
stage2_unit2_conv1 40 False
stage2_unit2_bn

In [0]:
################### Define the callbacks ###################

base_dir = '/content/drive/My Drive/EVA/Session20'


if not (os.path.exists(os.path.join(base_dir, 'weights'))):
    os.mkdir(os.path.join(base_dir, 'weights'))

filepath = os.path.join('/content/drive/My Drive/EVA/Session20/weights',
                        'weights-improvement-best.hdf5')

checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
def scheduler(epoch, lr):
    return round(lr * 1 / (1 + 0.050 * epoch), 10)

In [0]:
train_features.shape

(50000, 128, 128, 3)

In [0]:
model.compile(optimizer = keras.optimizers.SGD(lr=.1, momentum=.9, decay=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
start = time.time()
step_per_epoch_train = len(train_features) // 128
step_per_epoch_validation = len(test_features) // 128
# Train the model
model_info = model.fit_generator(datagen.flow(train_features, train_labels, batch_size = 128), epochs = 50,
                                 steps_per_epoch = step_per_epoch_train,
                                 validation_data = (test_features, test_labels), verbose=1,
                                 validation_steps = step_per_epoch_validation,
                                callbacks=[LearningRateScheduler(scheduler, verbose=1),checkpoint])
end = time.time()

print ("Model took %0.2f seconds to train"%(end - start))

Epoch 1/50

Epoch 00001: LearningRateScheduler setting learning rate to 0.1000000015.
390/390 [==============================] - 30s 77ms/step - loss: 0.9222 - acc: 0.7060 - val_loss: 1.0393 - val_acc: 0.6903

Epoch 00001: val_acc improved from -inf to 0.69030, saving model to /content/drive/My Drive/EVA/Session20/weights/weights-improvement-best.hdf5
Epoch 2/50

Epoch 00002: LearningRateScheduler setting learning rate to 0.0952380967.
390/390 [==============================] - 27s 70ms/step - loss: 0.4914 - acc: 0.8319 - val_loss: 0.5766 - val_acc: 0.8071

Epoch 00002: val_acc improved from 0.69030 to 0.80710, saving model to /content/drive/My Drive/EVA/Session20/weights/weights-improvement-best.hdf5
Epoch 3/50

Epoch 00003: LearningRateScheduler setting learning rate to 0.0865800882.
390/390 [==============================] - 27s 70ms/step - loss: 0.3724 - acc: 0.8708 - val_loss: 0.7345 - val_acc: 0.7861

Epoch 00003: val_acc did not improve from 0.80710
Epoch 4/50

Epoch 00004: Lear

In [0]:
model = keras.models.load_model(
    os.path.join('/content/drive/My Drive/EVA/Session20/weights/weights-improvement-best.hdf5'))
score = model.evaluate(test_features, test_labels, verbose=0)

In [0]:
print(score)

[0.6070131373941898, 0.8436]
